#RQ3. Define a new score!
Now it's your turn: build a new metric to rank MSc degrees.

Practically:



1.   The user will enter a text query. As a starting point, get the query-related documents by exploiting the search engine of Step 2.1.
2.   Once you have the documents, you need to sort them according to your new score. In this step, you won't have any more to take into account just the description field of the documents; you can use also the remaining variables in your dataset (or new possible variables that you can create from the existing ones or scrape again from the original web-pages). You must use a heap data structure (you can use Python libraries) for maintaining the top-k documents.

N.B.: You have to define a scoring function, not a filter!

The output, must contain:


*   courseName
*   universityName
*   description
*   URL
*   The new similarity score of the documents with respect to the query


Are the results you obtain better than with the previous scoring function? Explain and compare results.


In [ ]:
#Libraries

import pandas as pd
import os
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import pandas as pd
import os
import re
import requests
import numpy as np
import time
import folium
from folium import IFrame
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

We add a new Dataframe (2024 World University Rankings) to our existing one to build a scoring function

In [ ]:
# I used Google Drive for storing the data, You could find them in RQ 3 folder on github
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
qs_path = '/content/drive/MyDrive/ADM HW3/2024 World University Rankings.csv'
qs_database = pd.read_csv(qs_path)
qs_database.head()

,2024 RANK,2023 RANK,Institution Name,Country Code,Country,SIZE,FOCUS,RES.,AGE,STATUS,...,International Faculty Rank,International Students Score,International Students Rank,International Research Network Score,International Research Network Rank,Employment Outcomes Score,Employment Outcomes Rank,Sustainability Score,Sustainability Rank,Overall SCORE
0,1,1,Massachusetts Institute of Technology (MIT),US,United States,M,CO,VH,5.0,B,...,56,88.2,128,94.3,58,100.0,4,95.2,51,100.0
1,2,2,University of Cambridge,UK,United Kingdom,L,FC,VH,5.0,A,...,64,95.8,85,99.9,7,100.0,6,97.3,33=,99.2
2,3,4,University of Oxford,UK,United Kingdom,L,FC,VH,5.0,A,...,110,98.2,60,100.0,1,100.0,3,97.8,26=,98.9
3,4,5,Harvard University,US,United States,L,FC,VH,5.0,B,...,210,66.8,223,100.0,5,100.0,1,96.7,39,98.3
4,5,3,Stanford University,US,United States,L,FC,VH,5.0,B,...,78,51.2,284,95.8,44,100.0,2,94.4,63,98.1


Matching the collumn names on dataframes.

In [ ]:
# Get Masters degrees with tuition fees(aka M_d_w_tuit_f) as CSV file
M_d_w_tuit_f = pd.read_csv('/content/drive/MyDrive/ADM HW3/M_d_w_tuit_f.csv')

In [ ]:
# Get the dataset where was applied text-preprocessing to the column 'description' and adding the column 'fees_USD'
database_updated = pd.read_csv('/content/drive/MyDrive/ADM HW3/database_updated.csv')

In [ ]:
if 'fees_USD' in M_d_w_tuit_f.columns:
    # Replace 'fees' column in df_updated with 'fees_USD' column from M_d_w_tuit_f
    database_updated['fees'] = M_d_w_tuit_f['fees_USD']

In [ ]:
database_updated = database_updated.merge(qs_database, left_on='universityName', right_on='Institution Name', how='left')

We're streamlining our data by removing unnecessary columns, specifically dropping the 'Overall Score' column.

So we sets the stage for us to craft a custom scoring function.

In [ ]:
# Dropping columns from database_updated
columns_to_drop = ['Academic Reputation Rank', 'Employer Reputation Rank','Faculty Student Rank','Faculty Student Rank', 'Citations per Faculty Rank', 'International Faculty Rank', 'International Students Rank', 'International Research Network Rank', 'Employment Outcomes Rank',  'Sustainability Rank', 'Overall SCORE']
database_updated = database_updated.drop(labels=columns_to_drop, axis=1)
database_updated.head()

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,...,STATUS,Academic Reputation Score,Employer Reputation Score,Faculty Student Score,Citations per Faculty Score,International Faculty Score,International Students Score,International Research Network Score,Employment Outcomes Score,Sustainability Score
0,Future Manufacturing and Nanoscale Engineering...,University College London,Department of Mechanical Engineering,['Full Time'],Register your interest in graduate study at UC...,September,16819.44,MSc,1 year full time,London,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Marketing - MSc,University of Bristol,Faculty of Social Sciences and Law,['Full Time'],"This programme, accredited by both the Charter...",September,13090.60,MSc,1 year full-time,Bristol,...,A,74.9,84.7,47.3,64.7,85.5,86.8,84.4,39.6,81.3
2,Global Management of Natural Resources MSc,University College London,Department of Chemical Engineering,['Full Time'],Register your interest in graduate study at UC...,September,11735.89,MSc,1 year full time,London,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Business Analytics MSc,University of Stirling,"Business, Management and Law","['Full Time', 'Part Time']",Business Analytics focuses on understanding th...,"September, January",15.97,"PGCert,PGDip,MSc","12 months full-time, 24 months part-time",Stirling,...,A,11.5,10.2,10.5,34.7,71.9,83.5,14.4,14.8,79.8
4,Public Policy and Management MSc,King’s College London,King’s Business School,"['Full Time', 'Part Time']",Whether you are a social science graduate or a...,September,14280.66,MSc,"1 year FT, 2 years PT",London,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# print all column names of the DataFrame
print(database_updated.columns.tolist())

['courseName', 'universityName', 'facultyName', 'isItFullTime', 'description', 'startDate', 'fees', 'modality', 'duration', 'city', 'country', 'administration', 'url', 'description_lemmatized', 'courseName_lemmatized', 'universityName_lemmatized', 'city_lemmatized', 'fees_USD', '2024 RANK_x', '2023 RANK_x', 'Institution Name_x', 'Country Code_x', 'Country_x', 'SIZE_x', 'FOCUS_x', 'RES._x', 'AGE_x', 'STATUS_x', 'Academic Reputation Score_x', 'Employer Reputation Score_x', 'Faculty Student Score_x', 'Citations per Faculty Score_x', 'International Faculty Score_x', 'International Students Score_x', 'International Research Network Score_x', 'Employment Outcomes Score_x', 'Sustainability Score_x', '2024 RANK_y', '2023 RANK_y', 'Institution Name_y', 'Country Code_y', 'Country_y', 'SIZE_y', 'FOCUS_y', 'RES._y', 'AGE_y', 'STATUS_y', 'Academic Reputation Score_y', 'Academic Reputation Rank_x', 'Employer Reputation Score_y', 'Employer Reputation Rank_x', 'Faculty Student Score_y', 'Faculty Stude

Now lets start on scoring function.

In [ ]:
def calculate_score(document, query):

    # Initialize the score and define weights for different components
    weights = {
        'Academic Reputation': 0.15,
        'Employer Reputation': 0.15,
        'International Faculty': 0.1,
        'International Students': 0.1,
        'Citations per Faculty': 0.1,
        'Textual Relevance': 0.4
    }
    score = 0

    # Calculate scores for academic and employer reputation
    for component in ['Academic Reputation', 'Employer Reputation']:
        if not pd.isnull(document[f'{component} Score']):
            score += float(document[f'{component} Score']) / 100 * weights[component]

    # Calculate scores for international faculty and students
    for component in ['International Faculty', 'International Students']:
        if not pd.isnull(document[f'{component} Score']):
            score += float(document[f'{component} Score']) / 100 * weights[component]

    # Calculate score for citations per faculty
    if not pd.isnull(document['Citations per Faculty Score']):
        score += float(document['Citations per Faculty Score']) / 100 * weights['Citations per Faculty']

    # Calculate score for textual relevance
    query_terms = set(query.lower().split())
    description_terms = set(document['description_lemmatized'].lower().split())
    common_terms = query_terms.intersection(description_terms)
    score += len(common_terms) / len(query_terms) * weights['Textual Relevance']

    # Normalize the score to be out of 100
    total_weight = sum(weights.values())
    normalized_score = (score / total_weight) * 100

    return min(normalized_score, 100)  # Cap the score at 100


Re-Creating the inverted index

In [ ]:
# Get the vocabulary
vocabulary = pd.read_csv('/content/drive/MyDrive/ADM HW3/Vocabulary.csv')

In [ ]:
# Get the vocabulary_dict

vocabulary = set()

# Extract words from the 'description' column
for description in database_updated['description_lemmatized']:

     # Handling NaN/float values
     if isinstance(description, str):
        words = description.split()
        vocabulary.update(words)

# Convert the vocabulary to a dictionary with words as keys and their index as values
vocabulary_dict = {idx: word for idx, word in enumerate(vocabulary)}
print(vocabulary_dict)

{0: 'eibm', 1: 'environmentsextrem', 2: 'quintillion', 3: 'irelandthi', 4: 'pedigre', 5: 'psychopathologythi', 6: 'qualificationth', 7: 'pharmaceuticalsa', 8: '2024drive', 9: 'exportori', 10: 'nownowaday', 11: 'stock', 12: 'cbrn', 13: 'idealist', 14: 'taxonomi', 15: 'worldmsc', 16: 'theyv', 17: 'wilson', 18: 'guest', 19: 'interestar', 20: 'geoinformat', 21: 'theoryhow', 22: 'outputsw', 23: 'assessmenthow', 24: 'fame', 25: 'ioe', 26: 'studentship', 27: 'nationswheth', 28: 'award', 29: 'autonomi', 30: 'psychologyapprentic', 31: 'uncertain', 32: 'einen', 33: 'yeardur', 34: 'analysistheori', 35: 'aimsth', 36: 'andrang', 37: 'marketsdesign', 38: 'educationalist', 39: 'youdeliv', 40: 'internationallyrenown', 41: 'hygienist', 42: 'includinginvest', 43: 'replac', 44: 'descriptionmateri', 45: 'expertlydesign', 46: 'air', 47: 'practicefulltim', 48: 'consciou', 49: 'sustainabilitya', 50: 'schoolswith', 51: 'termperiod', 52: 'brunelanthropolog', 53: 'proactiv', 54: 'shanghaiin', 55: 'affili', 56: 

In [ ]:
# Create a dictionary to store word occurrences with row indices
result_dict = {word: [] for word in vocabulary}

# Iterate over rows to find occurrences
i = 0
for idx, description in enumerate(database_updated['description_lemmatized']):

    # Handling NaN/float values
    if isinstance(description, str):
        words = description.split()
        for word in words:
            if word in vocabulary:
                result_dict[word].append(idx)

words_frequency = {}

for index, word in vocabulary_dict.items():
    if word in result_dict:
        words_frequency[index] = result_dict[word]

words_frequency

# Create a DataFrame from the dictionary
inverted_index = pd.DataFrame(list(result_dict.items()), columns=['word', 'documents_IDs'])
inverted_index['documents_IDs'] = inverted_index['documents_IDs'].apply(lambda x: list(set(x)))
inverted_index.to_csv('inverted_index.csv', index=False)
inverted_index.head()


,word,documents_IDs
0,eibm,"[11546, 982]"
1,environmentsextrem,"[1498, 9197]"
2,quintillion,"[7028, 1886]"
3,irelandthi,"[11355, 11876]"
4,pedigre,"[9089, 1604, 4134, 5135, 6741, 7583]"


We're about to incorporate the calculation of similarity scores using Jaccard similarity.

 Following that, we'll arrange the master courses in our DataSet based on both their similarity scores and overall scores. This will refine our data arrangement for better analysis.

In [ ]:
# Text preprocessing function
def preprocess_text(text):

    # Handling NaN/float values
    if not isinstance(text, str):
        return ''
    # Converting the text to lower case
    text = text.lower()
    # Removing punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Breaking the text into words(tokens)
    words = nltk.word_tokenize(text)
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    # Stemming
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    # Joining the list of words to form the processed text
    processed_text = ' '.join(words)

    return processed_text

# Function to calculate Jaccard similarity score
def calculate_jaccard_similarity(query_terms, document_terms):
    intersection = query_terms.intersection(document_terms)
    union = query_terms.union(document_terms)
    if len(union) == 0:
        return 0
    return len(intersection) / len(union)

# Execute_query function
def execute_query(query, inverted_index, df):
    # Apply text preprocessing to the query
    processed_query = preprocess_text(query)
    query_words = processed_query.split()

    # Find document_IDs for each word in the query using the inverted index
    result_document_ids = []
    for word in query_words:
        if word in inverted_index['word'].values:
            # Retrieve and flatten the list of document IDs
            word_document_ids = inverted_index.loc[inverted_index['word'] == word, 'documents_IDs'].explode().tolist()
            result_document_ids.extend(word_document_ids)

    # Remove duplicate document_IDs and sort
    result_document_ids = sorted(set(result_document_ids))

    # Retrieve the rows corresponding to the document_IDs
    relevant_docs = df.loc[result_document_ids]

    # Process the descriptions and calculate similarity scores
    relevant_docs['processed_description'] = relevant_docs['description_lemmatized'].apply(preprocess_text)
    query_terms_set = set(processed_query.split())
    relevant_docs['similarity_score'] = relevant_docs['processed_description'].apply(
        lambda doc: calculate_jaccard_similarity(query_terms_set, set(doc.split()))
    )

    # Calculate relevance scores for each document
    relevant_docs['relevance_score'] = relevant_docs.apply(lambda doc: calculate_score(doc, query), axis=1)

    # Sort documents first by relevance score, then by similarity score
    sorted_results = relevant_docs.sort_values(by=['relevance_score', 'similarity_score'], ascending=[False, False])

    # Filter out additional courses from the same university, keeping only the most relevant one
    most_relevant_per_university = sorted_results.drop_duplicates(subset='universityName')

    # Select specific columns for the final output
    final_columns = ['courseName', 'universityName', 'description', 'url', 'fees_USD', 'city', 'similarity_score', 'relevance_score']
    final_results = most_relevant_per_university[final_columns]

    return final_results

In [ ]:
sample_query = 'advanced knowledge'
result = execute_query(sample_query, inverted_index, database_updated)
print(f'Number of results: {len(result)}')
result.head()

Number of results: 253


,courseName,universityName,description,url,fees_USD,city,similarity_score,relevance_score
468,Advanced Computational Methods for Aeronautics...,Imperial College London,This programme is suitable for applicants who ...,https://www.findamasters.com/masters-degrees/c...,0.0,London,0.042553,59.055
4473,MSc in Software Engineering,University of Oxford,The MSc in Software Engineering teaches the pr...,https://www.findamasters.com/masters-degrees/c...,0.0,Oxford,0.034483,58.700
3034,Doctor of Public Health,Harvard University,﻿Learn to translate knowledge into powerful re...,https://www.findamasters.com/masters-degrees/c...,0.0,Boston,0.062500,55.140
5189,MA/PGDip Heritage Studies,The University of Manchester,Course descriptionOur MA Heritage Studies mast...,https://www.findamasters.com/masters-degrees/c...,0.0,Manchester,0.025641,53.210
3983,Master of Science/Postgraduate Diploma in Envi...,The Hong Kong University of Science and Techno...,The program is meant to meet the needs of prac...,https://www.findamasters.com/masters-degrees/c...,0.0,Clear Water Bay,0.066667,47.935


When considering similarity scoring functions for search queries, both cosine and Jaccard similarity bring unique advantages to the table. Cosine similarity stands out for its speed and simplicity, evaluating the cosine of vectors in a multi-dimensional space, making it a popular choice due to its efficiency. On the flip side, Jaccard similarity measures shared attributes against total unique attributes, offering a distinct perspective on similarity that transcends mere angle calculation.

The introduction of TF-IDF to cosine similarity aims to enhance search precision by emphasizing unique terms. However, this refinement comes at the expense of time. The process significantly slows down due to the construction of the required inverted index. In contrast, the Jaccard similarity function skips this time-consuming step and cleverly integrates QS 2024 university rankings data, resulting in a more comprehensive and detailed range of outcomes. This approach not only swiftly identifies pertinent courses but also ranks them based on the prestige of the institutions—a pivotal aspect for many potential students.

Ultimately, while TF-IDF paired with cosine similarity might delve deeper into content, the Jaccard method's rapid delivery of comprehensive and contextualized outcomes—considering both search term matches and university reputation—proves more pragmatic. It strikes a balance, catering to users seeking a prompt yet insightful search experience by aligning course relevance with academic institution credibility.